# _ETL com Python_

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023_users.csv'), com uma lista de IDs de usuário do banco:
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.

In [4]:
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## **E**xtract
Extraindo a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, é feito uma requisição GET para obter os dados do usuário correspondente.

In [3]:
# Instalando a biblioteca pandas:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

df = pd.read_csv('SDW2023_users.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[3349]


In [15]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 3349,
    "name": "Louhen",
    "account": {
      "id": 3557,
      "number": "00001-99",
      "agency": "00099",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 3245,
      "number": "**** **** **** 0211",
      "limit": 10.0
    },
    "features": [],
    "news": []
  }
]


# **T**ransform
Utilizando a API do OpenAI GPT-3.5 para gerar uma mensagem de marketing personalizada para o usuário.

In [ ]:
# Instalando a API do ChatGPT:
%pip install openai

In [12]:
# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

openai_api_key = "token"

In [13]:
import openai

openai.api_key = openai_api_key

def generate_ai_news(user):
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em markting bancário."
      },
      {
          "role": "user",
          "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)"
      }
    ]
  )
  return completion.choices[0].message.content.strip('\"')

for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

AuthenticationError: Incorrect API key provided: token. You can find your API key at https://platform.openai.com/account/api-keys.

# **L**oad
Atualizando a lista de "news" do usuário na API com a nova mensagem gerada.

In [17]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Louhen updated? True!
